In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
import telebot
import requests
import matplotlib.pyplot as plt
from PIL import Image

bot = telebot.TeleBot("1799078640:AAFsJy-RhRDhZHNnXJiAObBqYIEj3xNnTr0")


@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Введи тикер акции, по которой хочешь узнать информацию! (пример: AAPL)")


@bot.message_handler(content_types = ['text'])
def get_info_3(message):
    #bot.send_message(message.chat.id, "Нативная интеграция. 🙃 \n\nРазвиваешь свой проект и не знаешь где купить отличный лого? Заходи на сайт http://nikitindesigner.tilda.ws/contacts!\n\nСкидка 15% по промокоду BIGKROLIK🤩")
    if len(message.text) <= 4 and len(message.text) >= 0: 
        try:
            stock_ticker = message.text
            bot.reply_to(message, f"Тикер: {stock_ticker}. Нейросеть обрабатывает информацию...")

            url = f'https://financialmodelingprep.com/api/v3/profile/{stock_ticker}?apikey=b59a97db01de231d304f7c4c20a0670d'
            result = requests.get(url)
            result_json = result.json()[0]
            cur = str(result_json["currency"])
            response = str(result_json["symbol"]) + '. Цена сейчас: ' + str(result_json["price"]) + ' ' + cur
            bot.send_message(message.chat.id, response)

            url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_ticker}&apikey=YD82B9BBTLJU86CO'
            result = requests.get(url)
            x = result.json()['Time Series (Daily)']
            c = []
            t = []
            for key in x.keys():
                t.append(key)
                c.append(float(x[key]['4. close']))
            c.reverse()
            t.reverse()
            plt.clf()
            plt.figure(figsize=(15, 6))
            plt.plot(t, c, label="Close Price - Day", color = 'blue');
            plt.xticks(size=7, rotation=90)
            plt.xlabel('Дата')
            plt.ylabel(f'{cur}')
            plt.title(f'Дневной график с начала 2021 года - {stock_ticker}', fontsize = 16, fontfamily = 'montserrat', fontweight = 'bold', alpha = 0.7, color = 'black')
            plt.grid(True);
            plt.savefig('saved_figure.png', dpi = 200)

            img = Image.open(r'saved_figure.png')
            bot.send_photo(message.chat.id, img)

            bot.send_message(message.chat.id, 'Ещё по тикеру?)')
        except:
            bot.send_message(message.chat.id, 'Онли американские акции)')
    else: 
        bot.send_message(message.chat.id, 'error, введи по-человечески')

bot.polling(none_stop = True)

In [5]:
YD82B9BBTLJU86CO

NameError: name 'YD82B9BBTLJU86CO' is not defined

In [ ]:
@bot.message_handler(commands=['about'])
def get_info_1(message):
    bot.send_message(message.chat.id, 'По какой акции интересует информация? (пример: AAPL)')
    
        
@bot.message_handler(commands=['recommendation'])
def get_info_2(message):
    bot.send_message(message.chat.id, 'По какой акции интересует информация? (пример: AAPL)')
    stock_ticker = message.text
    if len(message.text) <= 4 and len(message.text) >= 0: 
        url = f'https://financialmodelingprep.com/api/v3/rating/{stock_ticker}?apikey=b59a97db01de231d304f7c4c20a0670d'
        result = requests.get(url)
        result_json = result.json()[0]
        response = str(result_json["ratingScore"])+ ' out of 5.' + ' ' + str(result_json["ratingRecommendation"]) + '!'
        bot.send_message(message.chat.id, response)
    else:
        bot.send_message(message.chat.id, 'error')